In [3]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import models.conv_net as cnet

In [5]:
cnet.ConvNet(cnet.DEFAULT_PARAMS)

Layer 1,	type: conv2d,	filter: [4, 4],	FOV: 4
Layer 2,	type: pool,	filter: [2, 2],	FOV: 5
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Layer 3,	type: conv2d,	filter: [5, 5],	FOV: 13
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Layer 4,	type: pool,	filter: [2, 2],	FOV: 15
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.concat_v2().
Layer 5,	type: conv2d,	filter: [4, 4],	FOV: 27
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.

<models.conv_net.ConvNet instance at 0x7f6c5e52de60>

50